# UnSupervisedIOHMM

In [1]:
from __future__ import  division

import json
import warnings

import numpy as np
import pandas as pd

from IOHMM import UnSupervisedIOHMM
from IOHMM import OLS, DiscreteMNL, CrossEntropyMNL

warnings.simplefilter("ignore")

## Load speed data

In [9]:
data = pd.read_csv("data/TSLA.csv")
data.head()
data = data.dropna()

input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

#take only the first 100 rows
data = data[:100]

data.head()
#print(len(data))

,Open,High,Low,Close,Close-next-day
0,1.266667,1.666667,1.169333,1.592667,1.588667
1,1.719333,2.028000,1.553333,1.588667,1.464000
2,1.666667,1.728000,1.351333,1.464000,1.280000
3,1.533333,1.540000,1.247333,1.280000,1.074000
4,1.333333,1.333333,1.055333,1.074000,1.053333


# Example 1

## Set up a simple model manully

In [4]:
# set two hidden states
SHMM = UnSupervisedIOHMM(num_states=3, max_EM_iter=200, EM_tol=1e-6)

# This model has only one output which is modeled by a linear regression model
SHMM.set_models(model_emissions = [OLS()], 
                model_transition=CrossEntropyMNL(solver='lbfgs'),
                model_initial=CrossEntropyMNL(solver='lbfgs'))

# We don't set any covariates to this OLS model
SHMM.set_inputs(covariates_initial = ['Open', 'High', 'Low', 'Close'], covariates_transition = ['Open', 'High', 'Low', 'Close'], covariates_emissions = [['Open', 'High', 'Low', 'Close'],['Open', 'High', 'Low', 'Close'],['Open', 'High', 'Low', 'Close']])

SHMM.set_outputs([['Close-next-day']])

# we only have a list of one sequence.
SHMM.set_data([data])

## Start training

In [5]:
SHMM.train()

## See the training results

In [6]:
print(SHMM.model_initial)
print(SHMM.model_transition)
print(SHMM.model_emissions)

[<IOHMM.linear_models.CrossEntropyMNL object at 0x7fc2342e1f30>, <IOHMM.linear_models.CrossEntropyMNL object at 0x7fc2342e3a90>, <IOHMM.linear_models.CrossEntropyMNL object at 0x7fc2342e3b80>]
[[<IOHMM.linear_models.OLS object at 0x7fc1f6089f60>], [<IOHMM.linear_models.OLS object at 0x7fc1f608a1d0>], [<IOHMM.linear_models.OLS object at 0x7fc1f6089bd0>]]


In [7]:
print("Initial state distribution")
print(SHMM.model_initial.coef)

print("Transition matrix")
print(SHMM.model_transition[0].coef)
print(SHMM.model_transition[1].coef)
print(SHMM.model_transition[2].coef)

print("Emission model")
print(SHMM.model_emissions[0][0].coef)
print(SHMM.model_emissions[1][0].coef)
print(SHMM.model_emissions[2][0].coef)

Initial state distribution
[[ 0.74922054  0.94901294  1.24870114  0.87608829  1.19325882]
 [-0.37461027 -0.47450647 -0.62435057 -0.43804414 -0.59662941]
 [-0.37461027 -0.47450647 -0.62435057 -0.43804414 -0.59662941]]
Transition matrix
[[ -59.81167502   10.28288798   88.56955821  256.83150068 -312.58346279]
 [  61.15619245 -341.50542916  138.36268827 -275.86599604  436.51659652]
 [  -1.34451744  331.22254118 -226.93224648   19.03449536 -123.93313373]]
[[  5446.97304788   9321.16403109  -4161.17049986   4287.727949
  -13078.09772275]
 [  4494.91746505  -8085.54196443   2263.00394199  -5338.62599482
    7799.38091515]
 [ -9941.89051293  -1235.62206667   1898.16655787   1050.89804582
    5278.7168076 ]]
[[  241.01715178  -421.29952648   543.92324194   483.21508175
   -726.82981513]
 [  414.06457517  -749.07179485  -964.64578454   900.61194551
    620.55463385]
 [ -655.08172694  1170.37132133   420.72254261 -1383.82702726
    106.27518128]]
Emission model
[[-0.01836085  0.37753479 -0.427910

In [28]:
# The scale/dispersion of the OLS model for each hidden states
print(np.sqrt(SHMM.model_emissions[0][0].dispersion))
print(np.sqrt(SHMM.model_emissions[1][0].dispersion))
print(np.sqrt(SHMM.model_emissions[2][0].dispersion))

[[8.67150453]]
[[0.05989296]]
[[0.49065709]]
